# Setup

In [56]:
from tabulate import tabulate
import pandas as pd
from pathlib import Path
import json
import os
import altair as alt
from collections import defaultdict
from IPython.core.display import display, HTML
# alt.renderers.enable('html')

In [57]:
VERSION = '0.4.1'
def load_benchmark_results(VERSION):
    tables = defaultdict(lambda: defaultdict(dict))
    res_dir = 'results/%s' % VERSION
    results = []
    for fname in Path(res_dir).glob('*.json'):
        for l in open(fname):
            d = json.loads(l)
            res = {}
            # flattenning
            for k, v in d.items():
                if k == 'timings':
                    for tn, tv in v.items():
                        if tn == 'raw':
                            continue
                        name = '%s_time' % (tn)
                        res[name] = tv
                else:
                        res[k] = v
            res['shape_str'] = "x".join([str(x) for x in d['shape']])
            res['test_name'] = "%s%s" % (d['group'], d['name'])

            tables[res['test_name']][res['shape_str']][res['backend']] = res['avg_time']

            results.append(res)
    return pd.DataFrame(results), tables
df, tables = load_benchmark_results(VERSION)
df.head()

,avg_time,backend,evoflow_version,generations,group,input_size,max_time,min_time,name,num_runs,shape,shape_str,std_time,system,test_name,test_type,ts
0,0.002327,cupy,0.4.1,10,Shuffle,10000,0.005984,0.000000,1D,3,"[100, 100]",100x100,0.002618,"{'os': {'name': 'Windows', 'version': '10.0.18...",Shuffle1D,OP,1590167235
1,0.001662,cupy,0.4.1,10,RandomMutation,10000,0.002992,0.000997,1D,3,"[100, 100]",100x100,0.000940,"{'os': {'name': 'Windows', 'version': '10.0.18...",RandomMutation1D,OP,1590167235
2,0.002327,cupy,0.4.1,10,UniformCrossover,10000,0.004987,0.000997,1D,3,"[100, 100]",100x100,0.001881,"{'os': {'name': 'Windows', 'version': '10.0.18...",UniformCrossover1D,OP,1590167235
3,0.000332,cupy,0.4.1,10,SingleCrossover,10000,0.000997,0.000000,1D,3,"[100, 100]",100x100,0.000470,"{'os': {'name': 'Windows', 'version': '10.0.18...",SingleCrossover1D,OP,1590167235
4,0.001330,cupy,0.4.1,10,DualCrossover,10000,0.001995,0.000997,1D,3,"[100, 100]",100x100,0.000470,"{'os': {'name': 'Windows', 'version': '10.0.18...",DualCrossover1D,OP,1590167235


In [58]:
def htmlout(data):
    display(HTML(data))

def display_table(group_name):
    BACKENDS = ['numpy', 'tensorflow-cpu', 'tensorflow-gpu', 'cupy']
    headers = ['shape', 'numpy', 'tf-cpu', 'tf-gpu', 'cupy' ]
    # htmlout('<h3>%s</h3>' % group_name)
    for dim in ['1D', '2D', '3D']:
        prob_name = group_name + dim
        if prob_name in tables:
            raw = tables[prob_name]
            rows = []
            for shape, data in tables[prob_name].items():
                row = [shape]
                baseline = data['numpy']
                for b in  BACKENDS:
                    ts  = round(data[b], 3)
                    perf = round(baseline / (data[b] + 0.000001), 2)
                    val = "%s (%.2fx)" % (ts, perf)
                    row.append(val)
                rows.append(row)
            htmlout('<br/><b>%s</b>' % dim)
            htmlout(tabulate(rows, headers=headers, tablefmt='html'))


In [59]:
def display_chart(group_name):
    data = df[df['group']==group_name].sort_values('input_size')
    chart = alt.Chart(data, title='').mark_bar().encode(
            x=alt.X('backend', axis=alt.Axis(title='')), 
            y=alt.Y('avg_time', sort='-x', axis=alt.Axis(title='')),
            color=alt.Color('backend', legend=None), 
            column=alt.Column('shape_str', title='', sort=alt.EncodingSortField(field='input_size', order='ascending'))
            ).resolve_scale(y='independent')
    chart = chart.facet(column='name', spacing=50, title=group_name).configure_title(fontSize=20, offset=5, orient='top', anchor='middle')
    return chart

# End to End problems

In [60]:
display_chart('OneMax')

alt.FacetChart(...)

In [61]:
display_table('OneMax')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.008 (1.00x),0.017 (0.49x),0.028 (0.30x),0.027 (0.31x)
100x1000,0.033 (1.00x),0.056 (0.60x),0.055 (0.61x),0.025 (1.35x)
1000x1000,0.334 (1.00x),0.381 (0.88x),0.383 (0.87x),0.027 (12.57x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.008 (1.00x),0.019 (0.42x),0.032 (0.25x),0.031 (0.26x)
100x100x10,0.023 (1.00x),0.035 (0.66x),0.05 (0.46x),0.032 (0.72x)
100x100x100,0.374 (1.00x),0.378 (0.99x),0.415 (0.90x),0.035 (10.62x)


In [62]:
display_chart('TSP')

alt.FacetChart(...)

In [63]:
display_table('TSP')

shape,numpy,tf-cpu,tf-gpu,cupy
50x10,0.01 (1.00x),0.023 (0.43x),0.039 (0.26x),0.081 (0.12x)
250x50,0.028 (1.00x),0.029 (0.95x),0.043 (0.65x),0.112 (0.25x)
500x100,0.053 (1.00x),0.036 (1.50x),0.054 (0.99x),0.16 (0.33x)
1000x200,0.15 (1.00x),0.076 (1.98x),0.103 (1.46x),0.141 (1.06x)


# Evolution OPs

In [64]:
display_chart('RandomMutation')

alt.FacetChart(...)

In [65]:
display_table('RandomMutation')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.0 (1.00x),0.001 (0.33x),0.003 (0.11x),0.002 (0.20x)
100x1000,0.001 (1.00x),0.002 (0.67x),0.002 (0.57x),0.001 (1.33x)
1000x1000,0.012 (1.00x),0.018 (0.66x),0.017 (0.67x),0.001 (17.47x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (1.00x),0.001 (0.50x),0.001 (0.33x),0.001 (0.33x)
100x100x10,0.001 (1.00x),0.002 (0.40x),0.002 (0.40x),0.001 (0.67x)
100x100x100,0.013 (1.00x),0.019 (0.67x),0.018 (0.72x),0.001 (9.74x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.001 (1.00x),0.002 (0.40x),0.002 (0.40x),0.002 (0.40x)
100x100x10x10,0.01 (1.00x),0.015 (0.67x),0.016 (0.65x),0.002 (6.20x)
100x100x100x10,0.158 (1.00x),0.217 (0.73x),0.211 (0.75x),0.003 (52.87x)


In [66]:
display_chart('UniformCrossover')

alt.FacetChart(...)

In [67]:
display_table('UniformCrossover')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.001 (1.00x),0.001 (1.00x),0.001 (0.67x),0.002 (0.29x)
100x1000,0.002 (1.00x),0.002 (0.83x),0.002 (0.71x),0.001 (1.66x)
1000x1000,0.017 (1.00x),0.019 (0.93x),0.019 (0.91x),0.001 (17.32x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (1.00x),0.001 (0.33x),0.001 (0.33x),0.001 (0.25x)
100x100x10,0.001 (1.00x),0.001 (1.00x),0.002 (0.67x),0.001 (1.00x)
100x100x100,0.019 (1.00x),0.021 (0.87x),0.021 (0.87x),0.001 (13.99x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.001 (1.00x),0.001 (1.00x),0.002 (0.80x),0.002 (0.80x)
100x100x10x10,0.016 (1.00x),0.017 (0.92x),0.018 (0.87x),0.002 (9.59x)
100x100x100x10,0.209 (1.00x),0.237 (0.88x),0.233 (0.90x),0.004 (48.37x)


In [68]:
display_chart('SingleCrossover')

alt.FacetChart(...)

In [69]:
display_table('SingleCrossover')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.0 (1.00x),0.0 (332.28x),0.0 (1.00x),0.0 (1.00x)
100x1000,0.0 (0.00x),0.0 (0.00x),0.0 (0.00x),0.0 (0.00x)
1000x1000,0.003 (1.00x),0.005 (0.60x),0.005 (0.60x),0.0 (8.98x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (1.00x),0.0 (332.36x),0.0 (332.36x),0.0 (1.00x)
100x100x10,0.0 (1.00x),0.0 (332.36x),0.0 (1.00x),0.0 (332.36x)
100x100x100,0.002 (1.00x),0.005 (0.31x),0.005 (0.33x),0.001 (2.50x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.0 (0.00x),0.0 (0.00x),0.0 (0.00x),0.0 (0.00x)
100x100x10x10,0.002 (1.00x),0.005 (0.37x),0.005 (0.43x),0.0 (5.98x)
100x100x100x10,0.022 (1.00x),0.049 (0.45x),0.049 (0.45x),0.001 (21.98x)


In [70]:
display_chart('DualCrossover')

alt.FacetChart(...)

In [71]:
display_table('DualCrossover')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.0 (0.00x),0.001 (0.00x),0.001 (0.00x),0.001 (0.00x)
100x1000,0.0 (1.00x),0.0 (1.00x),0.001 (0.50x),0.001 (0.50x)
1000x1000,0.003 (1.00x),0.005 (0.56x),0.006 (0.53x),0.001 (4.49x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (1.00x),0.001 (0.50x),0.001 (0.50x),0.002 (0.20x)
100x100x10,0.0 (1.00x),0.001 (0.50x),0.001 (0.50x),0.001 (0.25x)
100x100x100,0.002 (1.00x),0.006 (0.28x),0.006 (0.28x),0.002 (1.00x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.0 (1.00x),0.001 (0.50x),0.001 (0.25x),0.002 (0.17x)
100x100x10x10,0.002 (1.00x),0.005 (0.37x),0.006 (0.35x),0.002 (0.86x)
100x100x100x10,0.022 (1.00x),0.05 (0.44x),0.05 (0.43x),0.003 (7.64x)


In [72]:
display_chart('Reverse')

alt.FacetChart(...)

In [73]:
display_table('Reverse')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.0 (1.00x),0.0 (1.00x),0.0 (1.00x),0.003 (0.12x)
100x1000,0.0 (1.00x),0.0 (1.00x),0.001 (0.50x),0.001 (0.25x)
1000x1000,0.001 (1.00x),0.006 (0.24x),0.005 (0.25x),0.001 (1.00x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (0.00x),0.001 (0.00x),0.001 (0.00x),0.002 (0.00x)
100x100x10,0.0 (0.00x),0.001 (0.00x),0.001 (0.00x),0.003 (0.00x)
100x100x100,0.001 (1.00x),0.006 (0.12x),0.006 (0.12x),0.003 (0.25x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.0 (1.00x),0.001 (0.33x),0.001 (0.25x),0.004 (0.08x)
100x100x10x10,0.001 (1.00x),0.005 (0.12x),0.006 (0.12x),0.004 (0.18x)
100x100x100x10,0.005 (1.00x),0.049 (0.11x),0.05 (0.11x),0.003 (1.60x)


In [74]:
display_chart('Shuffle')

alt.FacetChart(...)

In [75]:
display_table('Shuffle')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.0 (1.00x),0.001 (0.33x),0.001 (0.25x),0.002 (0.14x)
100x1000,0.002 (1.00x),0.002 (0.83x),0.002 (0.71x),0.001 (2.50x)
1000x1000,0.008 (1.00x),0.011 (0.71x),0.013 (0.63x),0.001 (6.00x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (1.00x),0.001 (0.33x),0.001 (0.33x),0.001 (0.25x)
100x100x10,0.001 (1.00x),0.001 (0.50x),0.001 (0.50x),0.001 (0.67x)
100x100x100,0.01 (1.00x),0.015 (0.64x),0.014 (0.69x),0.001 (7.24x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.001 (1.00x),0.001 (0.75x),0.001 (0.75x),0.002 (0.60x)
100x100x10x10,0.007 (1.00x),0.011 (0.62x),0.012 (0.56x),0.001 (5.00x)
100x100x100x10,0.136 (1.00x),0.18 (0.76x),0.172 (0.79x),0.003 (45.43x)


## Other

In [76]:
display_chart('Fittest')

alt.FacetChart(...)

In [77]:
display_table('Shuffle')

shape,numpy,tf-cpu,tf-gpu,cupy
100x100,0.0 (1.00x),0.001 (0.33x),0.001 (0.25x),0.002 (0.14x)
100x1000,0.002 (1.00x),0.002 (0.83x),0.002 (0.71x),0.001 (2.50x)
1000x1000,0.008 (1.00x),0.011 (0.71x),0.013 (0.63x),0.001 (6.00x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10,0.0 (1.00x),0.001 (0.33x),0.001 (0.33x),0.001 (0.25x)
100x100x10,0.001 (1.00x),0.001 (0.50x),0.001 (0.50x),0.001 (0.67x)
100x100x100,0.01 (1.00x),0.015 (0.64x),0.014 (0.69x),0.001 (7.24x)


shape,numpy,tf-cpu,tf-gpu,cupy
100x10x10x10,0.001 (1.00x),0.001 (0.75x),0.001 (0.75x),0.002 (0.60x)
100x100x10x10,0.007 (1.00x),0.011 (0.62x),0.012 (0.56x),0.001 (5.00x)
100x100x100x10,0.136 (1.00x),0.18 (0.76x),0.172 (0.79x),0.003 (45.43x)


In [78]:
sorted(set(df[df['test_type']=='Selection']['group']))


['Fittest']